<h1>Capstone Project - The Battle of Neighborhoods</h1>

<h2>A description of the problem and a discussion of the background.</h2>

<p>Someone wants to open a new restaurant in Moscow. Type of the restaurant - average prices, within the city. <b>The goal</b> is to recommend where they can open it.</p> 

<p><b>Stakeholders</b> - medium-sized businesses, looking for a stable source of profit but who do not have enough sources to create a high-class venue. The city center is full of restaurants, the competition is very high and the rental price is too high, so people are looking for alternative places.</p>

<h2>A description of the data and how it will be used to solve the problem.</h2>

<p><b>Data source</b> - Foursquare.</p>
<p>Find all venues within 5 km from the city center (Red Square). </p>
<p>Group venues by neighborhoods. </p>
<p> Add venues ranking by neighborhoods.</p>
<p>Cluster Neighborhoods.</p>
<p>Drop clusters with restaurants in top venues as there will be too high competition</p>
<p>Analyze rest clusters, find the one with the highest frequency of fashion shops and business centers in the top.</p>